In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import copy
import random
np.set_printoptions(suppress=True, threshold=np.inf)

## Data preparation

### Label encoder

In [4]:
demand_x = pd.read_csv("demand_x.csv")
demand_y = pd.read_csv("demand_y.csv")

In [5]:
date = demand_x.iloc[:,0]
demand_x = demand_x.iloc[:,1:]

In [7]:
labelencoder = LabelEncoder()
demand_x_encode = copy.deepcopy(demand_x)

In [8]:
demand_x_encode["Number"]=labelencoder.fit_transform(demand_x_encode["Number"])

In [9]:
demand_x_encode.Number

,Number,Month,t-3,t-2,t-1,t,t+1,t+2,t+3,t+4
0,3,5,20.0,0.0,0.0,5.5,5.0,39.5,0.0,95.0
1,3,5,0.0,0.0,5.5,5.0,39.5,0.0,95.0,14.5
2,3,6,0.0,5.5,5.0,39.5,0.0,95.0,14.5,0.0
3,3,6,5.5,5.0,39.5,0.0,95.0,14.5,0.0,0.0
4,3,6,5.0,39.5,0.0,95.0,14.5,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7814,19,10,0.0,0.0,0.0,155.0,0.0,0.0,257.0,0.0
7815,19,10,0.0,0.0,155.0,0.0,0.0,257.0,0.0,0.0
7816,19,10,0.0,155.0,0.0,0.0,257.0,0.0,0.0,0.0
7817,19,11,155.0,0.0,0.0,257.0,0.0,0.0,0.0,0.0


### Data shuffle

In [6]:
zip_list = list(zip(demand_x_encode.values.tolist(), demand_y.values.tolist()))
random.shuffle(zip_list)
demand_x_shuffle, demand_y_shuffle = zip(*zip_list)
demand_x_shuffle, demand_y_shuffle = np.array(demand_x_shuffle), np.array(demand_y_shuffle)

### Split data into training and testing

In [7]:
demand_x_shuffle_train = demand_x_shuffle[:int(demand_x_shuffle.shape[0]*0.8)]
demand_x_shuffle_test =  demand_x_shuffle[int(demand_x_shuffle.shape[0]*0.8):]
demand_y_shuffle_train = demand_y_shuffle[:int(demand_y_shuffle.shape[0]*0.8)]
demand_y_shuffle_test = demand_y_shuffle[int(demand_y_shuffle.shape[0]*0.8):]

In [8]:
print(demand_x_shuffle_train.shape)
print(demand_x_shuffle_test.shape)
print(demand_y_shuffle_train.shape)
print(demand_y_shuffle_test.shape)

(6255, 10)
(1564, 10)
(6255, 4)
(1564, 4)


## Model building

In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, Flatten, Input
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.1)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
tf.compat.v1.keras.backend.set_session(sess)

### ANN

#### -Building

In [10]:
def buildTwoLayerNN(input_shape):

    keras.backend.clear_session()
    model = Sequential()
    model.add(Dense(units=20, activation ="relu", input_shape=(input_shape,)))
    model.add(Dense(units=4)) 

    model.compile(optimizer="Adam", loss="mean_squared_error")

    return model

In [11]:
input_shape = demand_x_shuffle_train.shape[1]
model = buildTwoLayerNN(input_shape)

In [12]:
model.fit(demand_x_shuffle_train, demand_y_shuffle_train, epochs=300, batch_size=14, verbose=0)

#### -Forecast